In [1]:
import threading

In [6]:
def main():
    print(threading.active_count()) #显示总线程数
if __name__ == '__main__':
    main()

5


In [5]:
def main():
    print(threading.active_count())
    print(threading.enumerate()) # 查看每个线程，其中第一个为主线程
if __name__ == '__main__':
    main()

5
[<_MainThread(MainThread, started 3980)>, <Thread(Thread-4, started daemon 8824)>, <Heartbeat(Thread-5, started daemon 5324)>, <HistorySavingThread(IPythonHistorySavingThread, started 14672)>, <ParentPollerWindows(Thread-3, started daemon 4252)>]


In [7]:
def main():
    print(threading.active_count())
    print(threading.enumerate())
    print(threading.current_thread()) # 查看现在运行的程序是哪个线程
if __name__ == '__main__':
    main()

5
[<_MainThread(MainThread, started 3980)>, <Thread(Thread-4, started daemon 8824)>, <Heartbeat(Thread-5, started daemon 5324)>, <HistorySavingThread(IPythonHistorySavingThread, started 14672)>, <ParentPollerWindows(Thread-3, started daemon 4252)>]
<_MainThread(MainThread, started 3980)>


In [8]:
def thread_job():
    print('This is an added Thread, number is %s' % threading.current_thread())
    # 打印这个线程的名称

def main():
    added_thread = threading.Thread(target = thread_job) # 增加一个线程，目标是thread_job这个函数
    added_thread.start() # 运行这个线程
    
if __name__ == '__main__':
    main()

This is an added Thread, number is <Thread(Thread-6, started 7988)>


### join功能

In [9]:
import time
def thread_job():
    print('T1 start\n')
    for i in range(10):
        time.sleep(0.1)
    print('T1 finish\n')
    

def main():
    added_thread = threading.Thread(target = thread_job,name='T1') #也可以给线程命名为'T1' 
    added_thread.start() # 运行这个线程
    print('All done\n')
    
if __name__ == '__main__':
    main()

T1 start

All done

T1 finish



注意看结果的顺序，T1 finish实际是在1秒之后才print出来的，所以如果想所有线程都运行完后
再print上述例子中的All done，则需要用到join，如下

In [10]:
import time
def thread_job():
    print('T1 start\n')
    for i in range(10):
        time.sleep(0.1)
    print('T1 finish\n')
    

def main():
    added_thread = threading.Thread(target = thread_job,name='T1') 
    added_thread.start() 
    added_thread.join() # 加入这行则要等上面执行完后再执行后面操作
    print('All done\n')
    
if __name__ == '__main__':
    main()

T1 start

T1 finish

All done



如果增加一个线程，不用join，如下

In [11]:
import time
def thread_job():
    print('T1 start\n')
    for i in range(10):
        time.sleep(0.1)
    print('T1 finish\n')
    
def T2_job():   # 执行所消耗的时间要小于thread_job函数
    print('T2 start\n')
    print('T2 finish\n')
    
def main():
    added_thread = threading.Thread(target = thread_job,name='T1') 
    thread2 = threading.Thread(target = T2_job,name='T2')
    added_thread.start() 
    thread2.start()
    #added_thread.join() 
    print('All done\n')
    
if __name__ == '__main__':
    main()

T1 start

T2 start

T2 finish

All done

T1 finish



只加入thread_job的join

In [12]:
import time
def thread_job():
    print('T1 start\n')
    for i in range(10):
        time.sleep(0.1)
    print('T1 finish\n')
    
def T2_job():   # 执行所消耗的时间要小于thread_job函数
    print('T2 start\n')
    print('T2 finish\n')
    
def main():
    added_thread = threading.Thread(target = thread_job,name='T1') 
    thread2 = threading.Thread(target = T2_job,name='T2')
    added_thread.start() 
    thread2.start()
    added_thread.join() 
    print('All done\n')
    
if __name__ == '__main__':
    main()

T1 start

T2 start

T2 finish

T1 finish

All done



只加入T2_job的join

In [13]:
import time
def thread_job():
    print('T1 start\n')
    for i in range(10):
        time.sleep(0.1)
    print('T1 finish\n')
    
def T2_job():   # 执行所消耗的时间要小于thread_job函数
    print('T2 start\n')
    print('T2 finish\n')
    
def main():
    added_thread = threading.Thread(target = thread_job,name='T1') 
    thread2 = threading.Thread(target = T2_job,name='T2')
    added_thread.start() 
    thread2.start()
    thread2.join()
    print('All done\n')
    
if __name__ == '__main__':
    main()

T1 start

T2 start

T2 finish

All done

T1 finish



两个都同时join，则就是等待T1和T2都完成后再执行后续操作

In [14]:
import time
def thread_job():
    print('T1 start\n')
    for i in range(10):
        time.sleep(0.1)
    print('T1 finish\n')
    
def T2_job():   # 执行所消耗的时间要小于thread_job函数
    print('T2 start\n')
    print('T2 finish\n')
    
def main():
    added_thread = threading.Thread(target = thread_job,name='T1') 
    thread2 = threading.Thread(target = T2_job,name='T2')
    added_thread.start() 
    thread2.start()
    thread2.join()
    added_thread.join() 
    
    print('All done\n')
    
if __name__ == '__main__':
    main()

T1 start

T2 start

T2 finish

T1 finish

All done



### 多线程Queue功能
- 由于线程中没有办法返回一个值，所以需要用到queue

In [17]:
import threading
import time
from queue import Queue

def job(l,q):  # 参数l为list，列表
    for i in range(len(l)):
        l[i] = l[i]**2
    q.put(l)   # 这里不能用return，需要将return的东西放到q里面

def multithreading():
    q = Queue()  # 用它来记录job的返回值，来代替return的功能
    threads = [] # 将所有线程放入此列表中
    data = [[1,2,3],
            [3,4,5],
            [4,4,4],
            [5,5,5]]
    for i in range(4):  # 定义四个线程
        t = threading.Thread(target=job,args=(data[i],q)) # 注意：往job函数中加入参数的时候应该加到后面的args里面
        t.start()
        threads.append(t)
        
    for thread in threads:
        thread.join()
    
    results = [] # 储存最终结果
    for _ in range(4): # 这4次需要取出q里面的值
        results.append(q.get()) # q.get()每次会拿出一个列表
    print(results)
    
if __name__ == '__main__':
    multithreading()

[[1, 4, 9], [9, 16, 25], [16, 16, 16], [25, 25, 25]]


### GIL不一定有效率

为什么说 python 的多线程 threading 有时候并不是特别理想。最主要的原因是就是, Python 的设计上, 有一个必要的环节, 就是 Global Interpreter Lock (GIL)。 这个东西让 Python 还是一次性只能处理一个东西。

尽管Python完全支持多线程编程， 但是解释器的C语言实现部分在完全并行执行时并不是线程安全的。 实际上，解释器被一个全局解释器锁保护着，它确保任何时候都只有一个Python线程执行。 GIL最大的问题就是Python的多线程程序并不能利用多核CPU的优势 （比如一个使用了多个线程的计算密集型程序只会在一个单CPU上面运行）。

在讨论普通的GIL之前，有一点要强调的是GIL**只会影响到那些严重依赖CPU的程序**（比如计算型的）。**如果程序大部分只会涉及到I/O，比如网络交互，那么使用多线程就很合适**， 因为它们大部分时间都在等待。实际上，你完全可以放心的创建几千个Python线程，现代操作系统运行这么多线程没有任何压力，没啥可担心的。

### 测试GIL

创建一个 job, 分别用 threading和一般的方式执行这段程序. 并且创建一个 list 来存放要处理的数据. 在 Normal 的时候, 这个 list 扩展4倍, 在 threading 的时候, 建立4个线程, 并对运行时间进行对比.

In [18]:
import threading
from queue import Queue
import copy
import time

def job(l, q):
    res = sum(l)
    q.put(res)

def multithreading(l):
    q = Queue()
    threads = []
    for i in range(4):
        t = threading.Thread(target=job, args=(copy.copy(l), q), name='T%i' % i)
        t.start()
        threads.append(t)
    [t.join() for t in threads]
    total = 0
    for _ in range(4):
        total += q.get()
    print(total)

def normal(l):
    total = sum(l)
    print(total)

if __name__ == '__main__':
    l = list(range(1000000))
    s_t = time.time()
    normal(l*4)
    print('normal: ',time.time()-s_t)
    s_t = time.time()
    multithreading(l)
    print('multithreading: ', time.time()-s_t)

1999998000000
normal:  0.17555665969848633
1999998000000
multithreading:  0.15511584281921387


程序 threading 和 Normal 运行了一样多次的运算. 但是发现 threading 却没有快多少, 按理来说, 预期会要快3-4倍, 因为有建立4个线程, 但是并没有. 这就是其中的 GIL 在影响.

### 线程锁
- 期望第一个线程处理完的东西输入到第二个线程中

In [19]:
# 不加限制

import threading

def job1():
    global A  # 定义一个全局变量
    for i in range(10):
        A += 1
        print('job1',A)

def job2():
    global A
    for i in range(10):
        A += 10
        print('job2',A)
    
if __name__== '__main__':
    A=0
    t1=threading.Thread(target=job1)
    t2=threading.Thread(target=job2)
    t1.start()
    t2.start()
    t1.join()
    t2.join()

job1 1
job1 2
job1 3job2 13
job2 23
job2 33
job2 43
job2 53
job2 63
job2 73
job2 83
job2
job1 94
job1 95
job1 96
job1 97
job1 98
 93
job2 108
job1 109
job1 110


会发现print的东西很乱，下面演示让job1运算完后再运行job2

In [20]:
# 加上限制

import threading

def job1():
    global A,lock  # 将lock也设置为全局变量
    lock.acquire() # 开始lock住它
    for i in range(10):
        A += 1
        print('job1',A)
    lock.release() # 结束这个锁，在运行这之间的程序时，其它线程不会接触到它

def job2():
    global A,lock
    lock.acquire()
    for i in range(10):
        A += 10
        print('job2',A)
    lock.release()
    
if __name__== '__main__':
    lock = threading.Lock() # 定义一个锁
    A=0
    t1=threading.Thread(target=job1)
    t2=threading.Thread(target=job2)
    t1.start()
    t2.start()
    t1.join()
    t2.join()

job1 1
job1 2
job1 3
job1 4
job1 5
job1 6
job1 7
job1 8
job1 9
job1 10
job2 20
job2 30
job2 40
job2 50
job2 60
job2 70
job2 80
job2 90
job2 100
job2 110


这样两个线程之间就不会相互影响了